In [ ]:
import numpy as np
import pandas as pd
import os
from scipy.signal import iirnotch, filtfilt, butter, resample

# Load data
npz_file = np.load("/Users/saravanakumar.durais/Downloads/s01_Session3_run003.npz", allow_pickle=True)
data = npz_file['data']  # shape: (n_epochs, 2561, 91)

# Extract channel labels
header_row = data[0][0]
all_labels = [str(label).strip() for label in header_row[1:65]]

# Remove EOG
eeg_labels = [label for label in all_labels if label.lower() != 'eog']
eog_index = all_labels.index('EOG')
channel_indices = [i + 1 for i, label in enumerate(all_labels) if label.lower() != 'eog']

print(f"EEG channels after removing EOG: {len(eeg_labels)}")

# Concatenate all trials in order
all_data = []

for i in range(data.shape[0]):
    epoch = data[i][1:, channel_indices]  # shape: (2560, 63) - skipping the header row
    all_data.append(epoch.T)  # transpose to (channels, time)

# Final concatenated EEG data: shape (63, total_timepoints)
eeg_concat = np.hstack(all_data).astype(np.float32)


# --- Filtering and Downsampling ---
fs = 1024  # original sampling rate
target_fs = 256

# Notch filter at 50 Hz
f0 = 50.0  # Notch frequency
Q = 30.0   # Quality factor
b_notch, a_notch = iirnotch(f0, Q, fs)
eeg_notched = filtfilt(b_notch, a_notch, eeg_concat, axis=1)

# Bandpass filter between 0.5 and 80 Hz
lowcut = 0.5
highcut = 80.0
order = 4
b_band, a_band = butter(order, [lowcut / (fs / 2), highcut / (fs / 2)], btype='band')
eeg_filtered = filtfilt(b_band, a_band, eeg_notched, axis=1)

# Downsample to 256 Hz
num_samples = int(eeg_filtered.shape[1] * target_fs / fs)
eeg_downsampled = resample(eeg_filtered, num_samples, axis=1)

print(f"Final EEG shape after preprocessing: {eeg_downsampled.shape}")  # Should be (63, new_total_time)

# Save to CSV
output_csv = "/Users/saravanakumar.durais/Work/Speech_BCI/NEARBY/Raw_input1.csv"
pd.DataFrame(eeg_downsampled).to_csv(output_csv, header=False, index=False)
print(f"Saved processed EEG to: {output_csv}")
